<a href="https://colab.research.google.com/github/etccapital/Survey_PortfolioSelection/blob/master/PY_OLPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online Portfolio Selection
A Survey with Python Implementattion by ETC Quant




Problem Setting

**Basic Import Library**

In [0]:
import numpy as np
import pandas as pd
import scipy

**Strategy 1: Benchmarks** 

1.   **Buy and Hold Strategy (abbr. BAH)**

This strategy states that a portfolio manager will invest wealth among a pool of assets with an initial portfolio $\mathbf{b_{1}}$ and hold until the end. The final cumulative wealth achieved by a BAH strategy is initial portfolio weighted average of individual stocks' final wealth.
$$S_n(BAH(\mathbf{b_1})) = \mathbf{b_1} \cdot (\odot_{t=1}^{n}\mathbf{x_t})$$ Further, if $\mathbf{b_1}$ is the uniform portfolio $(\frac{1}{m}, \frac{1}{m}, \frac{1}{m}, ..., \frac{1}{m})$, then we called it as uniform BAH and this is often adopted as a *market* strategy to produce a market index. \\

2.   **Best Stock Strategy**

This is a special BAH strategy where we invest our wealth into the stock with best performance in hindsight, i.e., we find the best portfolio by maximizing the function: $$\mathbf{b} \cdot (\odot_{t=1}^{n}\mathbf{x_t})$$ where $\mathbf{b} \in \Delta m$, and we denote the portfolio that maximizes the above equation $\mathbf{b^{o}}$ As a result, the final cumulative wealth achieved by the Best (Stock) Strategy can be calculated as, $$S_n(Best) = \textbf{max} \ \mathbf{b} \cdot (\odot_{t=1}^{n}\mathbf{x_t}) = S_n(BAH(b^{o}))$$ \\

3.   **Constant Rebalanced Portfolios Strategy (abbr. CRP Strategy)**

This strategy rebalances the portfolio to a fixed portfolio $\mathbf{b}$ every period. The final cumulative portfolio wealth achieved by a CRP strategy after n periods is defined as, $$S_n(CRP(\mathbf{b})) = \prod_{t=1}^{n}\mathbf{b}^{T}\mathbf{x}_{t}$$ If the $\mathbf{b} = (\frac{1}{m}, \frac{1}{m}, ..., \frac{1}{m})$, then we called the strategy Uniform CRP, i.e, UCRP. \\
Now, if we maximize the function $$log \ S_n(CRP(\mathbf{b}))$$, and we get the portfolio that maximizes the function, $\mathbf{b}^{*}$. Such portfolio is called Best Constant Rebalanced Portfolio, abbr. BCRP. BCRP achieves a final cumulative portfolio wealth and corresponding exopential growth rate defined as follows, $$S_n(BCRP) = S_n(CRP(\mathbf{b}^{*}))$$, $$W_n(BCRP) = \frac{1}{n} log \ S_n(CRP(\mathbf{b}^{*}))$$ Note that BCRP strategy is a hindsight strategy, which can only be calculated with complete market sequences. Cover (1991) proved that BCRP exceeds the best stock, Value Line Index and Dow Jones Index. Moreover, BCRP is invariant under permutations of the price relative sequence, i.e., the order of $\mathbf{x}_{i}$ does not matter. \\

The following is the code for the Benchmark:






In [0]:
class BenchMark():
  def __init__(self, portfolio_name: str):
    self.portfolio_name = portfolio_name #Construct a portflio with a name!
  
  def ele_wise_prod(x: list, y: list): #x and y are with the same length
    s = []
    for i in range(len(x)):
      s.append(x[i] * y[i])
    return s
  def dot_product(a: list, b: list): #a and b are with the same length
    ans = 0
    for i in range(len(a)):
      ans += a[i] * b[i]
    return ans

  def BAH(b: list, x_t: list): #b is the inital portfolio as a vector, and x_t is the proportion increased at a window, as a list with each element as a list
    total_increase = []
    for i in range(len(b)):
      total_increase.append(1)
    for i in range(len(x_t)):
      new_increase = ele_wise_prod(total_increase, x_t[i])
      total_increase = new_increase[:]
    final_wealth = dot_product(b, total_increase)
    return final_wealth
  
  def Best_Stock(b_best: list, x_t: list): #We simply plug the optimized portfolio calculated by maths into BAH fnc
    return BAH(b_best, x_t)

  def BCRP(b_best: list, x_t: list): #We first calculate the best portflio by optimizing the log function and plug into the fnc to get the best result
    final_wealth = 1
    for i in range(len(x_t)):
      final_wealth = final_wealth * dot_product(b_best, x_t[i])
    return final_wealth 

**Strategy 2: Follow-the-Winner Approach**

The approach is characterized by increasing the relative weights of the more successful experts/stocks. However, rather than targeting market and best stock, algorithms in this category often aim to track the BCRP strategy. This aim is shown to be the optimal strategy in an i.i.d market [Cover and Thomas, 1991, Theorem 15.3.1].

1. **Universal Portfolios**

The basic idea of Universal Portfolios is to assign the captial into a single class of base experts, and the rest is analogous to BAH strategy. However, the difference is that base BAH expert is the strategy investing on a single stock and thus the number of experts is the same as that of stocks. i.e., BAH strategy buys the individual stocks and finally pools their individual wealth. While Follow-the-Winner category can be any strategy class that invests in any set of stocks in the market.


**Strategy 3: Follow-the-Loser Approach** 

The underlying assumption for the optimality of BCRP is that market is identical in distribution, which does not hold in real world data and thus often leads to inferior empirical performances. In contrast to Follow-the-Winner, this strategy transfer the assets from the outperforming assets to the underperforming ones. \\
 \\
The underlying assumptions is mean reversion[Bondt and Thaler 1985, Poterba and Summers 1988, Lo and Mackinlay 1990]. That is, the overperforming(underperforming) assets will perform poor(good) in the following periods. 




1.   **Anti-Correlation**

Anti-correlation strategy, a.k.a Anticor, assumes that the market follows the mean reversion principle. To exploit such property, it statistically makes bet on the consistency of postive lagged cross-correlation and negative auto-correlation.

Such, Anticor adopts logarithmic price relatives[Hull 2008] in two specific market windows in two specific market windows, that is, $y_1 = \log{x^{t-w}_{t-2w+1}}$ and
$y_2 = \log{x^{t}_{t-w+1}}$

It then calculates the cross-correlation matrix between $y_1$ and $y_2$.

$M_{cov(i,j)} = \frac{1}{w-1}(y_{1,i} - y_1)^T(y_{2,j} - y_2)$

$M_{cor(i,j)} = $


In [0]:
Class Anticor(Algo):
  """This strategy"""
  def __init__(self, window: int):
    """"""
    self.window = window
    pass
  
  def rebalance(self, x):
    port = x
    n, m = port.shape
    weights = 1/m*np.ones(port.shape)
    for t in range(n - 1):
      M = CORR[t, :, :]
      
    

2. **Passive Aggressive Mean Reversion, (abbr. PAMR)**

The strategy exploits the mean reversion property with the Passive Aggressive online learning. This strategy centred around the building of a loss function in order to reflect the mean reversion property. That is, the expected return based on last price relative is larger than a threshold, the loss will linearyly increase; otherwise, the loss is zero. In particular, we have a $\epsilon$-insensitve loss function for the $t^{th}$ period as,


$$l_\epsilon(b; x_t)=   \left\{
\begin{array}{ll}
      0 & bx_t\leq\epsilon \\
      b x_t - \epsilon  & otherwise \\
\end{array} 
\right.  $$

where $\epsilon$ is a sensitivity parameter that control the mean reversion threshold. Based on this loss function, PAMR passively maintain the last portfolio if loss is zero. Otherwise, it aggressively approaches a new portfolio that can force the loss zero. In short, PAMR obtains the rebalanced portfolio through the following optimization:

$$b_{t+1} = \arg\min_{b\in\Delta_m} \frac{1}{2} ||b-b_t||^2  s.t.  l_\epsilon(b;x_t)=0$$



3. **Confidence Weighted Mean Reversion, (abbr. CWMR)**

This algorithm futher exploit the second order portfolio information, the variance of portfolio weight. **(note: not price or price relative)** The basic idea is to model the portfolio vector as a multivariate Gaussian distribution with mean $\mu \in \mathbb{R^m}$ and the diagonal covariance matrix $\sum \in \mathbb{R^{m\times m}}$ which has non zero diagonal elements $\sigma^2$ and zero for off-diagonal elements.

In this mode, mean represents the knowledge for the portfolio, the diagonal covariance matrix term stands for the confidence we have in the corresponding portfolio mean. Then CWMR sequentially updates the mean and covariance matrix of the Gaussian distribution and draws portfolios from the distribution at the begging of the period. That is, the optimization to be solved is,

$$(\mu_{t+1}, \sum_{t+1}) = \arg\min_{\mu\in\Delta_m\sum} D_{KL}(\mathcal{N}(\mu, \sum) || \mathcal{N}  (\mu_t, \sum_t)) s.t. Pr[\mu \cdot x_t \le \epsilon] \ge \theta$$

To solve the optimization, we need to transform the optimization problem:

$$(\mu_{t+1}, \sum_{t+1})= \arg \min \frac{1}{2} (\log (\frac{\det\sum_t}{\det\sum} +Tr(\sum_t^{-1}\sum+(\mu_t-\mu))$$

s.t. $$\epsilon - \mu^Tx_t \ge \phi x_t^T \sum x_t$$

$$\mu^T1=1, \mu\ge 0$$


4. **Online Moving Average Reversion (abbr. OMAR or OLMAR)**

Note that the above strategy, PAMR and CWMR both have implicit assumption that single period means reversion. As convenient and effective as it is in theory, this is the core cause of failure in real dataset. OMRA was proposed to exploit statistical arbitrage opportunity on the foundation of *Moving Average Reversion*, a multiple-period mean reversion.

The basic intuition of OMAR is the observation that PAMR and CWMR implicitly predicts next last price, that is $p_{t+1} = p_{t-1}$, where p denotes the price vector corresponding the respective x. This extreme single period prediction creates large tail-risk. Instead, a multi-period mean reversion, explicitly predicts the next price vector as the moving average within the time window. This simple moving average, calculated by $MA_T = \frac{1}{w}\sum^t_{i=t-w+1}p_i$. Such, the next period price relative is:

$$x_{t+1} = \frac{MA_t(W)}{Pt}=\frac{1}{w}(1 + \frac{1}{x_t} + ... + \frac{1}{\odot_{i=0}^{w-2}x_{t-i}})$$

where $w$ is the window and $\odot$ represents element-wise product.


5. **Robust Median Reversion (abbr. RMR)** 

Noises and outliners are out of the scope of most existing mean reversion algorithms. They consequently are prone to estimation errors, and thereby leading to non-optimal portfolios and subsequent poor performances in practice. To tackle with noises and outliers, RMR is proposed to exploit mean reversion through a explicit estimation of next price vector at the end of $t^{th}$ period by robust $L_1$ - median estimator.

The optimization problem to be solved is,

$$\mu_{t+1} = \arg_{\mu} \min \sum^{w-1}_{i=0} ||p_{t-i} - \mu||$$

Such, the expected price relative with the $L_1$ median estimator is,

$$x_{t+1}(w) = \frac{L_1med_{t+1}(w)}{pt} = \frac{\mu_{t+1}}{pt}$$

Then, similar approach as in OLMAR is applied to learn the optimal portfolio. This algorithms outperforms on most of the state-of-the-art datasets.

**Strategy 4: Pattern Matching based Approaches**

The assumption of Pattern Maching based Approaches is that a historical behavior of market would highly possible reappear in the future. The Algorithm 3 demonstrates how to identify a historical market pattern. Then the optimization problem is to optimize the portfolio that  maximizes the expeted return with using a history market pattern which is similar to the upcoming market behavior. The optimization is

$$b_{t+1} = \mathop{\arg\max}_{b\in\Delta_m}\prod_{i\in C(x_1^t)}b\cdot x_i$$



1. Nonparameteric Kernel-based Log-Optimal Strategy $B^K$\
The similarity of two market windows is defined by its Euclidean distance. Therefore samples can be selected by limiting the similarity.
$$C_k(x_1^t,w)=\{w<i<t+1:||x^t_{t-w+1}-x^{i-1}_{i-w}||\leq \frac cl\}$$
where $c$ and $l$ are the thresholds for limiting the amount of similar samples.



2. Nonparameteric Nearest Neighbor Log-Optimal Strategy $B^{NN}$\
Choosing multiple market windows rather than choosing one. Samples can be selected within $l$ nearest neighbor of the latest market window.
$$C_N(x_1^t,w)=\{w<i<t+1:x^{i-1}_{i-w}\ is\ among\ the\ l\ NNs\ of\ x^t_{t-w+1}\}$$
where $l$ is a parameter which limit the number of preceding market windows.



3. Correlation-Driven Nonparametric Learning Strategy $CORN$\
The similarity of two market windows is defined by its correlation coefficient,
$$C_C(x_1^t,w)=\{w<i<t+1:\frac {cov(x^{i-1}_{i-w},x^t_{t-w+1})}{std(x^{i-1}_{i-w})std(x^t_{t-w+1})} \geq \rho\}$$
where $\rho$ is a threshold parameter.